In [1]:
import gurobipy as gp
from gurobipy import GRB
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import colorsys
import pickle

In [2]:
def print_nice(width, height, x,y, dx,dy, scale=10):
    img = Image.new('RGB', (width*scale, height*scale), color = (255, 255, 255))
    draw = ImageDraw.Draw(img)
    shape = [(x[i]*scale, (height-y[i])*scale, (x[i]+dx[i])*scale, (height-y[i]-dy[i])*scale) for i in range(len(x))]
    for i in range(len(x)):
        (r,g,b) = colorsys.hsv_to_rgb(((i+1)*(360/len(x)))/360, 0.5, 1.0)
        draw.rectangle(shape[i], fill=(int(255*r), int(255*g), int(255*b)))
        
    for i in range(len(x)):
        draw.text((x[i]*scale+scale/1.8, (height-y[i]-1)*scale), str(i), fill="black")
    display(img)
    print(f"Final height: {max([y[i]+dy[i] for i in range(len(x))])}")

def remove_outofboud(x,y,dx,dy, width, height):
    i=0
    while i < len(x):
        if x[i]+dx[i] > width or y[i] > height:
            del x[i]
            del y[i]
            del dx[i]
            del dy[i]
            i -= 1
        i += 1
    return (x, y, dx, dy)

def show_result(data):
    if len(data) == 2:
        rec_pos, rec_dim = data
        print("Positions:", rec_pos)
        print("Dimension:", rec_dim)
    else:
        rec_pos, rec_dim, rec_rot = data
        print("Positions:", rec_pos)
        print("Dimension:", rec_dim)
        print("Rotations:", rec_rot)
        rec_dim = [rec_dim[i] if rec_rot[i]==0 else (rec_dim[i][1], rec_dim[i][0]) for i in range(len(rec_dim))]
        
    width = max([rec_pos[i][0] + rec_dim[i][0] for i in range(len(rec_pos))])
    height = max([rec_pos[i][1] + rec_dim[i][1] for i in range(len(rec_pos))])
    if rec_pos == None:
        return
    x = [rec_pos[i][0] for i in range(len(rec_pos))]
    y = [rec_pos[i][1] for i in range(len(rec_pos))]
    dx = [rec_dim[i][0] for i in range(len(rec_dim))]
    dy = [rec_dim[i][1] for i in range(len(rec_dim))]
    (x, y, dx, dy) = remove_outofboud(x, y, dx, dy, width, height)
    
    if len(data) == 2:
        return (x, y, dx, dy, rec_pos, rec_dim)
    else:
        return (x, y, dx, dy, rec_pos, rec_dim, rec_rot)
        
    
    return (x, y, dx, dy) 
    #print_nice(width, height, x, y, dx, dy, 10)

In [3]:


def problem_no_rotation(width, dx, dy):
    n = len(dx)
    #best_height = sum(intdx[i]*dy[i]) for i in range(n) // width
    best_height = sum([int(dx[i])*int(dy[i]) for i in range(n)]) // width
    print(f"Best height: {best_height}")
    safe_height = int(best_height*1.3)
    m = gp.Model("mip1")
    m.setParam('TimeLimit', 5*60)
    
    X = [m.addVar(vtype=GRB.INTEGER, name="x_"+str(i)) for i in range(n)]
    Y = [m.addVar(vtype=GRB.INTEGER, name="y_"+str(i)) for i in range(n)]
    H_s = [m.addVar(vtype=GRB.INTEGER, name="hs_"+str(i)) for i in range(n)]
    for i in range(n):
        m.addConstr(X[i] <= width - dx[i], "width_"+str(i))
        m.addConstr(H_s[i] - Y[i] == dy[i], "top_edge_"+str(i))
   
    # Non overlap
    for i in range(n):
        for j in range(i+1,n):
            
            b = [m.addVar(vtype=GRB.BINARY, name="b_"+str(k)+"_"+str(i)+"_"+str(j)) for k in range(4)]
            m.addConstr(sum(b) >= 1, "diffn_b_"+str(i)+"_"+str(j))
            
            m.addConstr(X[i] - X[j] <= -dx[i] + (dx[i] + width)*(1-b[0]))
            m.addConstr(X[j] - X[i] <= -dx[j] + (dx[j] + width)*(1-b[1]))
            m.addConstr(Y[i] - Y[j] <= -dy[i] + (dy[i] + safe_height)*(1-b[2]))
            m.addConstr(Y[j] - Y[i] <= -dy[j] + (dy[j] + safe_height)*(1-b[3]))
    
    # Height aka objective function
    H = m.addVar(vtype=GRB.INTEGER, name="height")
    b = [m.addVar(vtype=GRB.BINARY, name="b_height_"+str(i)) for i in range(n)]
    m.addConstr(sum(b) == 1, "height_sum_b")
    for i in range(n):
        m.addConstr(H >= H_s[i], "height_less_Y_"+str(i))
        m.addConstr(H <= H_s[i] + (safe_height-min(dy))*(1-b[i]), "height_less_Y_"+str(i))
    m.addConstr(H >= best_height, "best_height")
    m.setObjective(H, GRB.MINIMIZE)
    m.optimize()
   
    
    return ([(int(X[i].X), int(Y[i].X)) for i in range(n)], [(dx[i], dy[i]) for i in range(n)])

In [4]:
def problem_rotation(width, dx, dy):
    n = len(dx)
    best_height = sum([dx[i]*dy[i] for i in range(n)]) // width
    
    print(f"Best height: {best_height}")
    safe_height = int(best_height*1.3)
    m = gp.Model("mip2")
    
    X = [m.addVar(vtype=GRB.INTEGER, name="x_"+str(i)) for i in range(n)]
    Y = [m.addVar(vtype=GRB.INTEGER, name="y_"+str(i)) for i in range(n)]
    R = [m.addVar(vtype=GRB.BINARY, name="r_"+str(i)) for i in range(n)]
    H_s = [m.addVar(vtype=GRB.INTEGER, name="hs_"+str(i)) for i in range(n)]
    for i in range(n):
        m.addConstr(X[i] + (1-R[i])*dx[i] + R[i]*dy[i] <= width, "width_"+str(i))
        m.addConstr(H_s[i] - Y[i]*(1-R[i]) + (dy[i]-dx[i])*R[i] == dy[i], "top_edge_"+str(i))
   
    # Non overlap
    for i in range(n):
        for j in range(i+1,n):
            
            b = [m.addVar(vtype=GRB.BINARY, name="b_"+str(k)+"_"+str(i)+"_"+str(j)) for k in range(4)]
            m.addConstr(sum(b) >= 1, "diffn_b_"+str(i)+"_"+str(j))
            
#             m.addConstr((X[i] - X[j])*(1-R[i]) <= (-dx[i] + (dx[i] + width)*(1-b[0]))*(1-R[i]) )
#             m.addConstr((X[j] - X[i])*(1-R[j]) <= (-dx[j] + (dx[j] + width)*(1-b[1]))*(1-R[j]) )
#             m.addConstr((Y[i] - Y[j])*(1-R[i]) <= (-dy[i] + (dy[i] + safe_height)*(1-b[2]))*(1-R[i]) )
#             m.addConstr((Y[j] - Y[i])*(1-R[j]) <= (-dy[j] + (dy[j] + safe_height)*(1-b[3]))*(1-R[j]) )
                        
#             m.addConstr((X[i] - X[j])*(R[i]) <= (-dy[i] + (dy[i] + width)*(1-b[0]))*(R[i]) )
#             m.addConstr((X[j] - X[i])*(R[j]) <= (-dy[j] + (dy[j] + width)*(1-b[1]))*(R[j]) )
#             m.addConstr((Y[i] - Y[j])*(R[i]) <= (-dx[i] + (dx[i] + safe_height)*(1-b[2]))*(R[i]) )
#             m.addConstr((Y[j] - Y[i])*(R[j]) <= (-dx[j] + (dx[j] + safe_height)*(1-b[3]))*(R[j]) )
            
            m.addConstr((X[i] - X[j]) <= width - b[0]*dx[i] + b[0]*R[i]*dx[i] - b[0]*R[i]*dy[i] - b[0]*width )
            m.addConstr((X[j] - X[i]) <= width - b[1]*dx[j] + b[1]*R[j]*dx[j] - b[1]*R[j]*dy[j] - b[1]*width )
            m.addConstr((Y[i] - Y[j]) <= safe_height - b[2]*dy[i] + b[2]*R[i]*dy[i] - b[2]*R[i]*dx[i] - b[2]*safe_height )
            m.addConstr((Y[j] - Y[i]) <= safe_height - b[3]*dy[j] + b[3]*R[j]*dy[j] - b[3]*R[j]*dx[j] - b[3]*safe_height )
            
    
    # Height aka objective function
    H = m.addVar(vtype=GRB.INTEGER, name="height")
    b = [m.addVar(vtype=GRB.BINARY, name="b_height_"+str(i)) for i in range(n)]
    m.addConstr(sum(b) == 1, "height_sum_b")
    
    for i in range(n):
        m.addConstr(H >= H_s[i], "height_less_Y_"+str(i))
        m.addConstr(H <= H_s[i] + (safe_height-min(dy))*(1-b[i]), "height_less_Y_"+str(i))
        
    # Implied constraint
    m.addConstr(H >= best_height, "best_height")
    m.addConstr(H<= safe_height, "safe_height")
    m.setObjective(H, GRB.MINIMIZE)
    print("Compiled")
    m.optimize()
    #timeout=300
    m.setParam("timeout", 5*60)
    
    return ([(int(X[i].X), int(Y[i].X)) for i in range(n)], [(dx[i], dy[i]) for i in range(n)], [int(R[i].X) for i in range(n)])

In [5]:
# code for output
import math
import time

for i in range(1, 41):

    file_name = "C:/Users/singh/Documents/CDMO MIP/instances/instances/ins-" + str(i) + ".txt"
    buf = open(file_name)
    W = int(buf.readline())
    number_of_blocks = int(buf.readline())

    width = []
    height = []
    width_rotated = []
    height_rotated = []

    for line in buf:
        tmp = line.split()
        tmp = [int(x) for x in tmp]
        width.append(tmp[0])
        height.append(tmp[1])
        width_rotated.append(tmp[1])
        height_rotated.append(tmp[0])

    H = int(math.ceil(sum([width[i] * height[i] for i in range(number_of_blocks)]) / W))

    start = time.time()
    show_result(problem_no_rotation(W, width, height))
    end = time.time()

    print(file_name + ' ' + "{:.2f}".format(end - start))

    #cornerx, cornery = converter_sat_coord(s.model(), px, py, number_of_blocks)

    #print(os.getcwd())
    out_file = 'C:/Users/singh/Documents/CDMO MIP/instances/output/out-{}.txt'.format(i)
    out_buf = open(out_file, 'w')
    out_buf.write('Plate Width '+ str(W) + ' Height ' + str(H) + '\n')
    out_buf.write('Number of chips ' + str(number_of_blocks) + '\n')

    for (a, b) in zip(width, height):
        out_buf.write('Plate width ' + str(W) + ' x-coordinate of chip ' + str(width) + ' y-coordinate ' + str(height) + '\n')

Best height: 8
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-26
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 48 rows, 41 columns and 133 nonzeros
Model fingerprint: 0xbc88693f
Variable types: 0 continuous, 41 integer (28 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 8.0000000
Presolve removed 15 rows and 12 columns
Presolve time: 0.00s
Presolved: 33 rows, 29 columns, 98 nonzeros
Variable types: 0 continuous, 29 integer (22 binary)

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 8 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+

In [6]:
# code for output-rotation
import multiprocessing
import time

T=0
Time_list=[]
import math
import time
for i in range(1, 41):

    file_name = "C:/Users/singh/Documents/CDMO MIP/instances/instances/ins-" + str(i) + ".txt"
    buf = open(file_name)
    W = int(buf.readline())
    number_of_blocks = int(buf.readline())

    width = []
    height = []

    for line in buf:
        tmp = line.split()
        tmp = [int(x) for x in tmp]
        width.append(tmp[0])
        height.append(tmp[1])
    print(width)
    print("\n")
    print(height)


    start = time.time()
    (x, y, dx, dy) = show_result(problem_rotation(W, width, height))
    
#     p = multiprocessing.Process(target=show_result, name="show_result", args=(40,))
#     p.start()

#     # Wait 10 seconds for foo
#     time.sleep(10)
    
#     if p.is_alive():
        
        

#     # Terminate foo
#         p.terminate()
#         p.join()
    
    
    
    end = time.time()
    T=end-start
    Time_list.append(T)
    
print(Time_list)

[3, 3, 5, 5]


[3, 5, 3, 5]
Best height: 8
Compiled
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 21 rows, 45 columns and 56 nonzeros
Model fingerprint: 0x1437ad7f
Model has 28 quadratic constraints
Variable types: 0 continuous, 45 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [3e+00, 1e+01]
Presolve added 26 rows and 0 columns
Presolve time: 0.05s
Presolved: 59 rows, 57 columns, 188 nonzeros
Presolved model has 8 SOS constraint(s)
Variable types: 0 continuous, 57 integer (36 binary)
Found heuristic solution: objective 10.0000000
Found heuristic solution: objective 8.0000000

Explored 0 nodes (0 simplex iterations) in 0.09 seconds (0.00 work units)
Thread count wa

ValueError: too many values to unpack (expected 4)

In [44]:

# code for output-rotation
import multiprocessing
import time

T=0
Time_list=[]
import math
import time
for i in range(1, 41):

    file_name = "C:/Users/singh/Documents/CDMO MIP/instances/instances/ins-" + str(i) + ".txt"
    buf = open(file_name)
    W = int(buf.readline())
    number_of_blocks = int(buf.readline())

    width = []
    height = []

    for line in buf:
        tmp = line.split()
        tmp = [int(x) for x in tmp]
        width.append(tmp[0])
        height.append(tmp[1])
    print(width)
    print("\n")
    print(height)


    start = time.time()
    (x, y, dx, dy, positions , dimensions , rotations) = show_result(problem_rotation(W, width, height))
    
    print('Hi I am Pratiksha and this is a list', positions)
    
    end = time.time()
    T=end-start
    Time_list.append(T)
    
    out_file = 'C:/Users/singh/Documents/CDMO MIP/instances/output_rotation/out-{}.txt'.format(i)
    out_buf = open(out_file, 'w')
    
    #out_buf.write('Positions- \n')
    with open(out_file, 'w') as output_file:
        for p in positions:
            print(p, file=output_file)
    
    #out_buf.write('Dimensions- \n')
    with open(out_file, 'w') as output_file:
        for d in dimensions:
            print(d, file=output_file)
                 
    #out_buf.write('Rotations- \n')
    with open(out_file, 'w') as output_file:
        for r in rotations:
            print(r, file=output_file)
    
    
print(Time_list)











[3, 3, 5, 5]


[3, 5, 3, 5]
Best height: 8
Compiled
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 21 rows, 45 columns and 56 nonzeros
Model fingerprint: 0x1437ad7f
Model has 28 quadratic constraints
Variable types: 0 continuous, 45 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [3e+00, 1e+01]
Presolve added 26 rows and 0 columns
Presolve time: 0.00s
Presolved: 59 rows, 57 columns, 188 nonzeros
Presolved model has 8 SOS constraint(s)
Variable types: 0 continuous, 57 integer (36 binary)
Found heuristic solution: objective 10.0000000
Found heuristic solution: objective 8.0000000

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count wa